# Problem **1**

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.patches as patches

# Create the figure and axis
fig, ax = plt.subplots(figsize=(10, 6))

# Draw the background rectangle
background = patches.Rectangle((0, 0), 10, 6, facecolor='forestgreen')
ax.add_patch(background)

# Draw the red circle
circle = patches.Circle((4.5, 3), radius=2, facecolor='red')
ax.add_patch(circle)

# Set the limits
ax.set_xlim(0, 10)
ax.set_ylim(0, 6)


# Display the flag
plt.show()


# **Problem** 2
Run this code in any other IDE not in Colab

In [ ]:
import pygame
import sys
from math import sin, cos

# Define the colors
WHITE = (255, 255, 255)
BLACK = (0, 0, 0)
RED = (255, 0, 0)
GREEN = (0, 255, 0)
BLUE = (0, 0, 255)
YELLOW = (255, 255, 0)
CYAN = (0, 255, 255)
MAGENTA = (255, 0, 255)

# Define the vertices of the 3D object
cube_vertices = [
    (-2, -2, -2),
    (-2, 2, -2),
    (2, 2, -2),
    (2, -2, -2),
    (-2, -2, 2),
    (-2, 2, 2),
    (2, 2, 2),
    (2, -2, 2)
]
# Define the edges of the 3D object
cube_edges = [
    (0, 1),
    (1, 2),
    (2, 3),
    (3, 0),
    (4, 5),
    (5, 6),
    (6, 7),
    (7, 4),
    (0, 4),
    (1, 5),
    (2, 6),
    (3, 7)
]

# Define the faces of the 3D object with colors
cube_faces = [
    ((0, 1, 2, 3), RED),
    ((3, 2, 6, 7), GREEN),
    ((7, 6, 5, 4), BLUE),
    ((4, 5, 1, 0), YELLOW),
    ((5, 6, 2, 1), CYAN),
    ((7, 4, 0, 3), MAGENTA)
]

# Initialize pygame
pygame.init()

# Set up the display
display_width = 800
display_height = 600
display = pygame.display.set_mode((display_width, display_height))
pygame.display.set_caption("Hidden Surface Elimination")

# Define the camera position
camera_pos = (0, 0, -5)

# Define the projection plane
projection_plane_z = 0

# Define the viewing angle
fov = 90

# Define the rotation angles
rotation_angle_x = 0
rotation_angle_y = 0

# Rotate the cube vertices
def rotate(vertices, angle_x, angle_y):
    rotated_vertices = []
    for vertex in vertices:
        x, y, z = vertex

        # Rotate around X-axis
        y_rot = y * cos(angle_x) - z * sin(angle_x)
        z_rot = y * sin(angle_x) + z * cos(angle_x)

        # Rotate around Y-axis
        x_rot = x * cos(angle_y) + z_rot * sin(angle_y)
        z_rot = -x * sin(angle_y) + z_rot * cos(angle_y)

        rotated_vertices.append((x_rot, y_rot, z_rot))
    return rotated_vertices

# Project the 3D vertices onto the 2D projection plane
def project(vertices):
    projected_vertices = []
    for vertex in vertices:
        x, y, z = vertex
        if z == camera_pos[2]:
            z += 0.01  # Avoid division by zero
        distance = camera_pos[2] - z
        factor = fov / distance
        x_proj = int(x * factor + display_width / 2)
        y_proj = int(y * factor + display_height / 2)
        projected_vertices.append((x_proj, y_proj))
    return projected_vertices

# Draw the 3D object on the display
def draw(vertices, edges, faces):
    # Clear the display
    display.fill(BLACK)

    # Project the vertices
    projected_vertices = project(vertices)

    # Draw the faces
    for face, color in faces:
        vertices_list = [projected_vertices[vertex] for vertex in face]
        pygame.draw.polygon(display, color, vertices_list)

    # Draw the edges
    #for edge in edges:
        #vertex1, vertex2 = projected_vertices[edge[0]], projected_vertices[edge[1]]
        #pygame.draw.line(display, WHITE, vertex1, vertex2, 3)

    # Draw the camera marker
    camera_marker_pos = project([camera_pos])[0]
    pygame.draw.circle(display, WHITE, camera_marker_pos, 5)

    # Draw the text "camera" next to the camera marker
    font = pygame.font.SysFont(None, 20)
    text = font.render("Camera", True, WHITE)
    text_rect = text.get_rect()
    text_rect.x = camera_marker_pos[0] + 10
    text_rect.y = camera_marker_pos[1] - 10
    display.blit(text, text_rect)

    # Update the display
    pygame.display.update()

# Create a slider class
class Slider:
    def __init__(self, x, y, width, height, min_val, max_val, initial_val, color, handle_color):
        self.x = x
        self.y = y
        self.width = width
        self.height = height
        self.min_val = min_val
        self.max_val = max_val
        self.value = initial_val
        self.color = color
        self.handle_color = handle_color

    def draw(self):
        pygame.draw.rect(display, self.color, (self.x, self.y, self.width, self.height))
        handle_x = self.x + int((self.value - self.min_val) / (self.max_val - self.min_val) * self.width)
        pygame.draw.circle(display, self.handle_color, (handle_x, self.y + self.height // 2), self.height // 2)

    def update(self, mouse_x):
        if mouse_x < self.x:
            self.value = self.min_val
        elif mouse_x > self.x + self.width:
            self.value = self.max_val
        else:
            relative_x = mouse_x - self.x
            self.value = self.min_val + (self.max_val - self.min_val) * relative_x / self.width

# Create a slider for X-axis rotation
slider_x = Slider(50, 550, 700, 20, -3.14, 3.14, rotation_angle_x, WHITE, RED)

# Create a slider for Y-axis rotation
slider_y = Slider(50, 580, 700, 20, -3.14, 3.14, rotation_angle_y, WHITE, RED)

# Game loop
clock = pygame.time.Clock()
running = True
while running:
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False
        elif event.type == pygame.MOUSEMOTION:
            mouse_x, mouse_y = event.pos
            if pygame.mouse.get_pressed()[0]:
                slider_x.update(mouse_x)
                slider_y.update(mouse_x)

    # Rotate the cube
    rotation_angle_x = slider_x.value
    rotation_angle_y = slider_y.value
    cube_vertices_rotated = rotate(cube_vertices, rotation_angle_x, rotation_angle_y)

    # Sort the faces based on their distance from the camera
    sorted_faces = sorted(cube_faces, key=lambda face: max(cube_vertices_rotated[vertex][2] for vertex in face[0]), reverse=True)

    # Draw the cube
    draw(cube_vertices_rotated, cube_edges, sorted_faces)

    # Draw the sliders
    slider_x.draw()
    slider_y.draw()

    # Limit the frame rate
    clock.tick(60)

# Quit the program
pygame.quit()
sys.exit()


# Problem **3**

In [ ]:
#Problem 3.1
import numpy as np
import matplotlib.pyplot as plt

def translate(points, tx, ty):
    translated_points = points + np.array([tx, ty])
    return translated_points

# Create a square
points = np.array([[4, 4],
                   [6, 4],
                   [6, 6],
                   [4, 6],
                   [4, 4]])

# Translation
translated_points = translate(points, 2, 3)

# Plotting the original and translated squares
plt.figure(figsize=(8, 4))

plt.subplot(1, 2, 1)
plt.plot(points[:, 0], points[:, 1], 'bo-')
plt.title('Original Square')
plt.xlim(-10, 10)
plt.ylim(-10, 10)
plt.gca().set_aspect('equal', adjustable='box')

plt.subplot(1, 2, 2)
plt.plot(translated_points[:, 0], translated_points[:, 1], 'ro-')
plt.title('Translated Square')
plt.xlim(-10, 10)
plt.ylim(-10, 10)
plt.gca().set_aspect('equal', adjustable='box')

plt.tight_layout()
plt.show()

print("Original Points:\n", points)
print("Translated Points:\n", translated_points)


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import math

def rotate(points, angle):
    rad_angle = math.radians(angle)
    rotation_matrix = np.array([[math.cos(rad_angle), -math.sin(rad_angle)],
                                [math.sin(rad_angle), math.cos(rad_angle)]])
    rotated_points = np.dot(points, rotation_matrix)
    return rotated_points

# Create a square
points = np.array([[-1, -1],
                   [1, -1],
                   [1, 1],
                   [-1, 1],
                   [-1, -1]])

# Rotation
rotated_points = rotate(points, 45)

# Plotting the original and rotated squares
plt.figure(figsize=(8, 4))

plt.subplot(1, 2, 1)
plt.plot(points[:, 0], points[:, 1], 'bo-')
plt.title('Original Square')
plt.xlim(-10, 10)
plt.ylim(-10, 10)
plt.gca().set_aspect('equal', adjustable='box')

plt.subplot(1, 2, 2)
plt.plot(rotated_points[:, 0], rotated_points[:, 1], 'go-')
plt.title('Rotated Square')
plt.xlim(-10, 10)
plt.ylim(-10, 10)
plt.gca().set_aspect('equal', adjustable='box')

plt.tight_layout()
plt.show()

print("Original Points:\n", points)
print("Rotated Points:\n", rotated_points)


In [ ]:
#Problem_3.3
import numpy as np
import matplotlib.pyplot as plt

def scale(points, sx, sy):
    scaled_points = points * np.array([sx, sy])
    return scaled_points

# Create a square
points = np.array([[-1, -1],
                   [1, -1],
                   [1, 1],
                   [-1, 1],
                   [-1, -1]])

# Scaling
scaled_points = scale(points, 2, 2)

# Plotting the original and scaled squares
plt.figure(figsize=(8, 4))

plt.subplot(1, 2, 1)
plt.plot(points[:, 0], points[:, 1], 'bo-')
plt.title('Original Square')
plt.xlim(-10, 10)
plt.ylim(-10, 10)
plt.gca().set_aspect('equal', adjustable='box')

plt.subplot(1, 2, 2)
plt.plot(scaled_points[:, 0], scaled_points[:, 1], 'ro-')
plt.title('Scaled Square')
plt.xlim(-10, 10)
plt.ylim(-10, 10)
plt.gca().set_aspect('equal', adjustable='box')

plt.tight_layout()
plt.show()

print("Original Points:\n", points)
print("Scaled Points:\n", scaled_points)


# Problem **4**

In [ ]:
#Problem 4.1
import matplotlib.pyplot as plt
import numpy as np
from mpl_toolkits.mplot3d import Axes3D

# Generate sphere coordinates
theta = np.linspace(0, 2 * np.pi, 100)
phi = np.linspace(0, np.pi, 50)
theta, phi = np.meshgrid(theta, phi)
radius = 1
x = radius * np.sin(phi) * np.cos(theta)
y = radius * np.sin(phi) * np.sin(theta)
z = radius * np.cos(phi)

# Plot the sphere in 2D
plt.subplot(121)
plt.title('Sphere (2D)')
plt.xlabel('X')
plt.ylabel('Y')
plt.axis('equal')
plt.grid(True)
plt.plot(x, y, 'b.')

# Plot the sphere in 3D
ax = plt.subplot(122, projection='3d')
ax.set_title('Sphere (3D)')
ax.set_xlabel('X')
ax.set_ylabel('Y')
ax.set_zlabel('Z')
ax.plot_surface(x, y, z, cmap='viridis')

# Display the plots
plt.tight_layout()
plt.show()


In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from mpl_toolkits.mplot3d import Axes3D

# Generate ellipsoid coordinates
theta = np.linspace(0, 2 * np.pi, 100)
phi = np.linspace(0, np.pi, 50)
theta, phi = np.meshgrid(theta, phi)
a = 2  # x-axis radius
b = 1  # y-axis radius
c = 1  # z-axis radius
x = a * np.sin(phi) * np.cos(theta)
y = b * np.sin(phi) * np.sin(theta)
z = c * np.cos(phi)

# Calculate the maximum radius of the ellipsoid
max_radius = max(a, b, c)

# Plot the ellipsoid in 2D
plt.subplot(121)
plt.title('Ellipsoid (2D)')
plt.xlabel('X')
plt.ylabel('Y')
plt.axis('equal')
plt.grid(True)
plt.plot(x, y, 'b.')

# Plot the ellipsoid in 3D
ax = plt.subplot(122, projection='3d')
ax.set_title('Ellipsoid (3D)')
ax.set_xlabel('X')
ax.set_ylabel('Y')
ax.set_zlabel('Z')

# Set limit of all three axes to the maximum radius
ax.set_xlim(-max_radius, max_radius)
ax.set_ylim(-max_radius, max_radius)
ax.set_zlim(-max_radius, max_radius)

ax.plot_surface(x, y, z, cmap='viridis')

# Display the plots
plt.tight_layout()
plt.show()


In [ ]:
#Problem 4.3
import matplotlib.pyplot as plt
import numpy as np
from mpl_toolkits.mplot3d import Axes3D

# Generate torus coordinates
theta = np.linspace(0, 2 * np.pi, 100)
phi = np.linspace(0, 2 * np.pi, 50)
theta, phi = np.meshgrid(theta, phi)
R = 2  # Major radius (distance from center of tube to center of torus)
r = 1  # Minor radius (radius of the tube)
x = (R + r * np.cos(theta)) * np.cos(phi)
y = (R + r * np.cos(theta)) * np.sin(phi)
z = r * np.sin(theta)

# Plot the torus in 2D
plt.subplot(121)
plt.title('Torus (2D)')
plt.xlabel('X')
plt.ylabel('Y')
plt.axis('equal')
#plt.grid(False)
plt.plot(x, y, 'b.')

# Plot the torus in 3D
ax = plt.subplot(122, projection='3d')
ax.set_title('Torus (3D)')
ax.set_xlabel('X')
ax.set_ylabel('Y')
ax.set_zlabel('Z')
ax.plot_surface(x, y, z, cmap='viridis')

# Display the plots
plt.tight_layout()
plt.show()


# Problem **5**

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

# Set up the parameters
a = 100  # x-axis radius
b = 100  # y-axis radius
c = 100  # z-axis radius
n1 = 2   # exponent for x-axis
n2 = 2   # exponent for y-axis
n3 = 2   # exponent for z-axis

# Set up the transformation variables
theta = np.linspace(0, 2 * np.pi, 100)
phi = np.linspace(0, np.pi, 100)

# Calculate the vertices of the superellipsoid
x = np.outer(np.cos(theta), np.sin(phi))
y = np.outer(np.sin(theta), np.sin(phi))
z = np.outer(np.ones(np.size(theta)), np.cos(phi))
x = a * np.sign(x) * np.power(np.abs(x), n1)
y = b * np.sign(y) * np.power(np.abs(y), n2)
z = c * np.sign(z) * np.power(np.abs(z), n3)

# Plot the Superellipsoid in 2D and 3D side by side
fig, axes = plt.subplots(1, 2, figsize=(12, 6))

# Plot the Superellipsoid in 2D
axes[0].contourf(x, y, levels=100, cmap='viridis')
axes[0].set_xlabel('X')
axes[0].set_ylabel('Y')
axes[0].set_title('Superellipsoid (2D)')

# Plot the Superellipsoid in 3D
axes[1] = fig.add_subplot(122, projection='3d')
axes[1].plot_surface(x, y, z, cmap='viridis')
axes[1].set_xlabel('X')
axes[1].set_ylabel('Y')
axes[1].set_zlabel('Z')
axes[1].set_title('Superellipsoid (3D)')

plt.tight_layout()
plt.show()


In [ ]:
#Problem 5.1
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

# Set up the parameters
a = 100  # x-axis radius
b = 100  # y-axis radius
n = 0.5  # exponent

# Generate the points of the superellipse
theta = np.linspace(0, 2 * np.pi, 100)
x = a * np.sign(np.cos(theta)) * np.power(np.abs(np.cos(theta)), 2/n)
y = b * np.sign(np.sin(theta)) * np.power(np.abs(np.sin(theta)), 2/n)

# Generate the 3D points of the superellipse
phi = np.linspace(-np.pi/2, np.pi/2, 100)
theta_3d = np.linspace(0, 2 * np.pi, 100)
theta_3d, phi_3d = np.meshgrid(theta_3d, phi)
x_3d = a * np.sign(np.cos(theta_3d)) * np.power(np.abs(np.cos(theta_3d)), 2/n) * np.cos(phi_3d)
y_3d = b * np.sign(np.sin(theta_3d)) * np.power(np.abs(np.sin(theta_3d)), 2/n) * np.cos(phi_3d)
z_3d = np.power(np.abs(np.sin(phi_3d)), 2/n)

# Create a figure with subplots for 2D and 3D
fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(10, 5))

# Plot the Superellipse in 2D
axes[0].plot(x, y)
axes[0].set_xlabel('X')
axes[0].set_ylabel('Y')
axes[0].set_title('Superellipse (2D)')

# Plot the Superellipse in 3D
axes[1] = fig.add_subplot(122, projection='3d')
axes[1].plot_surface(x_3d, y_3d, z_3d, cmap='viridis')
axes[1].set_xlabel('X')
axes[1].set_ylabel('Y')
axes[1].set_zlabel('Z')
axes[1].set_title('Superellipse (3D)')

plt.tight_layout()
plt.show()


# Problem **6**

In [ ]:
#Problem 6
import matplotlib.pyplot as plt
import numpy as np

def compute_bezier_curve(control_points, num_points):
    t = np.linspace(0, 1, num_points)
    n = len(control_points) - 1
    curve_points = np.zeros((num_points, 2))

    for i in range(num_points):
        for j in range(n + 1):
            curve_points[i] += control_points[j] * binomial_coefficient(n, j) * (1 - t[i]) ** (n - j) * t[i] ** j

    return curve_points

def binomial_coefficient(n, i):
    return np.math.factorial(n) / (np.math.factorial(i) * np.math.factorial(n - i))

# Define control points
control_points = np.array([[0, 0], [2, 5], [5, 3], [7, 8]])

# Compute Bezier curve points
num_points = 100
curve_points = compute_bezier_curve(control_points, num_points)

# Plot the Bezier curve
plt.plot(control_points[:, 0], control_points[:, 1], 'ro-', label='Control Points')
plt.plot(curve_points[:, 0], curve_points[:, 1], 'b-', label='Bezier Curve')
plt.legend()
plt.xlabel('X')
plt.ylabel('Y')
plt.title('Bezier Curve')
plt.grid(True)
plt.show()


# Problem **7**

In [ ]:
import matplotlib.pyplot as plt

def draw_line(x0, y0, xn, yn):
    # Step 1: Calculate differences and absolute values
    delta_x = abs(xn - x0)
    delta_y = abs(yn - y0)

    # Step 2: Calculate the decision parameter P_k
    p = 2 * delta_y - delta_x

    # Step 3: Compare P_k for necessary calculations
    x_step = 1 if x0 < xn else -1
    y_step = 1 if y0 < yn else -1

    # Generate the points of the line
    x = x0
    y = y0
    points = []
    for _ in range(delta_x - 1):
        # Append the current point to the list
        points.append((x, y))

        # Step 4: Calculate P_k for the next iteration
        if p < 0:
            p = p + 2 * delta_y
            x = x + x_step
            y = y
        else:
            p = p + 2 * delta_y - 2 * delta_x
            x = x + x_step
            y = y + y_step

        # Increment x-coordinate


    return points

# Define the line coordinates
x0, y0 = 1, 1
xn, yn = 8, 5

# Call the line drawing function
line_points = draw_line(x0, y0, xn, yn)

# Print the points
print("Line Points:")
for point in line_points:
    print(point)

# Plot the line
x_coords = []
y_coords = []
for point in line_points:
    x_coords.append(point[0])
    y_coords.append(point[1])
plt.plot(x_coords, y_coords, marker='o')
plt.xlabel('X')
plt.ylabel('Y')
plt.title('Bresenham Line Drawing')
plt.grid(True)
plt.show()


# Problem **8**

In [ ]:
import matplotlib.pyplot as plt

def plot_points(x, y, xc, yc):
    """
    Plot the points of the circle using symmetry.
    """
    plt.plot(xc + x, yc + y, 'ro')
    plt.plot(xc - x, yc + y, 'ro')
    plt.plot(xc + x, yc - y, 'ro')
    plt.plot(xc - x, yc - y, 'ro')
    plt.plot(xc + y, yc + x, 'ro')
    plt.plot(xc - y, yc + x, 'ro')
    plt.plot(xc + y, yc - x, 'ro')
    plt.plot(xc - y, yc - x, 'ro')

def midpoint_circle(xc, yc, r):
    """
    Draw a circle using the Midpoint Circle Drawing Algorithm.
    """
    x = 0
    y = r
    p = 1 - r  # Initial decision parameter

    # Plot the first point
    plot_points(x, y, xc, yc)

    while x < y:
        if p < 0:
            x = x + 1
            y = y
            p = p + 2 * x + 1
        else:
            x = x + 1
            y = y - 1
            p = p - 2 * (x - y) + 1

        # Plot the points
        plot_points(x, y, xc, yc)

    plt.axis('equal')  # Set aspect ratio to be equal
    plt.show()

# Example usage
x_center = 0
y_center = 0
radius = 5

midpoint_circle(x_center, y_center, radius)


# **Problem_9**

In [ ]:
import matplotlib.pyplot as plt

# Define the region codes
INSIDE = 0  # 0000
LEFT = 1    # 0001
RIGHT = 2   # 0010
BOTTOM = 4  # 0100
TOP = 8     # 1000

# Define the clipping window boundaries
x_min = 50
y_min = 50
x_max = 200
y_max = 200

# Function to compute the region code for a point (x, y)
def compute_region_code(x, y):
    code = INSIDE

    if x < x_min:
        code |= LEFT
    elif x > x_max:
        code |= RIGHT

    if y < y_min:
        code |= BOTTOM
    elif y > y_max:
        code |= TOP

    return code

# Function to clip a line using Cohen-Sutherland algorithm
def clip_line(x1, y1, x2, y2):
    # Compute the region codes for the two endpoints
    code1 = compute_region_code(x1, y1)
    code2 = compute_region_code(x2, y2)

    output = f"Initial Line: ({x1}, {y1}) to ({x2}, {y2})"
    output += f"\nInitial Region Codes: Point 1 - {code1}, Point 2 - {code2}\n"

    while True:
        # If both endpoints are inside the clip window, accept the line
        if code1 == 0 and code2 == 0:
            output += "Line completely inside the clipping window."
            return (x1, y1, x2, y2), output

        # If the bitwise AND of both codes is not zero, the line is outside the window
        if code1 & code2 != 0:
            output += "Line completely outside the clipping window."
            return None, output

        # Select an endpoint outside the clip window
        code = code1 if code1 != 0 else code2

        # Find the intersection point
        if code & TOP:
            x = x1 + (x2 - x1) * (y_max - y1) / (y2 - y1)
            y = y_max
        elif code & BOTTOM:
            x = x1 + (x2 - x1) * (y_min - y1) / (y2 - y1)
            y = y_min
        elif code & RIGHT:
            y = y1 + (y2 - y1) * (x_max - x1) / (x2 - x1)
            x = x_max
        elif code & LEFT:
            y = y1 + (y2 - y1) * (x_min - x1) / (x2 - x1)
            x = x_min

        output += f"Intersection Point: ({x}, {y})\n"

        # Update the endpoint outside the clip window with the intersection point
        if code == code1:
            x1, y1 = x, y
            code1 = compute_region_code(x1, y1)
        else:
            x2, y2 = x, y
            code2 = compute_region_code(x2, y2)

        output += f"Updated Line: ({x1}, {y1}) to ({x2}, {y2})"
        output += f"\nUpdated Region Codes: Point 1 - {code1}, Point 2 - {code2}\n"

# Define the line coordinates
x1, y1 = 30, 240
x2, y2 = 220, 120

# Clip the line
result, output = clip_line(x1, y1, x2, y2)

# Print the output
print(output)

# Plotting the original line and the clipped line
plt.plot([x1, x2], [y1, y2], 'r', label='Original Line')
if result is not None:
    x1, y1, x2, y2 = result
    plt.plot([x1, x2], [y1, y2], 'g', label='Clipped Line')
plt.xlim(0, 250)
plt.ylim(0, 250)
plt.axhline(y_min, color='k', linestyle='--')
plt.axhline(y_max, color='k', linestyle='--')
plt.axvline(x_min, color='k', linestyle='--')
plt.axvline(x_max, color='k', linestyle='--')
plt.legend()
plt.show()


# Problem **10**


In [ ]:
import matplotlib.pyplot as plt

# Function to perform polygon clipping
def clip(subjectPolygon, frame):
    # Function to check if a point is inside an edge
    def inside(p, cp1, cp2):
        return (cp2[0] - cp1[0]) * (p[1] - cp1[1]) > (cp2[1] - cp1[1]) * (p[0] - cp1[0])

    # Function to calculate the intersection point
    def intersection(cp1, cp2, s, e):
        dc = [cp1[0] - cp2[0], cp1[1] - cp2[1]]
        dp = [s[0] - e[0], s[1] - e[1]]
        n1 = cp1[0] * cp2[1] - cp1[1] * cp2[0]
        n2 = s[0] * e[1] - s[1] * e[0]
        n3 = 1.0 / (dc[0] * dp[1] - dc[1] * dp[0])
        return [(n1 * dp[0] - n2 * dc[0]) * n3, (n1 * dp[1] - n2 * dc[1]) * n3]

    # Make a copy of the subject polygon
    outputList = subjectPolygon.copy()
    cp1 = frame[-1]  # Last point of the clip polygon
    for cp2 in frame:
        inputList = outputList
        outputList = []
        s = inputList[-1]  # Last point of the subject polygon
        for e in inputList:
            if inside(e, cp1, cp2):  # Check if point e is inside the edge formed by cp1 and cp2
                if not inside(s, cp1, cp2):  # Check if point s is outside the edge formed by cp1 and cp2
                    outputList.append(intersection(cp1, cp2, s, e))  # Add intersection point to output
                outputList.append(e)  # Add point e to output
            elif inside(s, cp1, cp2):  # Check if point s is inside the edge formed by cp1 and cp2
                outputList.append(intersection(cp1, cp2, s, e))  # Add intersection point to output
            s = e
        cp1 = cp2

    return outputList

# Create a subject polygon in the shape of a star
subjectPolygon = [(-2, 0.3), (-0.3, 0.3), (0, 3), (0.3, 0.3), (2, 0), (0.3, -0.3), (0, -2), (-0.3, -0.3)]

# Define the rectangular frame
frame= [(-1.5, -1.5), (1.5, -1.5), (1.5, 1.5), (-1.5, 1.5)]

# Call the clip function to obtain the output polygon
outputPolygon = clip(subjectPolygon, frame)

# Extract the x and y coordinates of the output polygon
output_x = [point[0] for point in outputPolygon]
output_y = [point[1] for point in outputPolygon]

# Plot the subject polygon
subject_x = [point[0] for point in subjectPolygon]
subject_y = [point[1] for point in subjectPolygon]
plt.plot(subject_x + [subject_x[0]], subject_y + [subject_y[0]], 'b-', label='Subject Polygon')

# Plot the clip polygon
clip_x = [point[0] for point in frame]
clip_y = [point[1] for point in frame]
plt.plot(clip_x + [clip_x[0]], clip_y + [clip_y[0]], 'r-', label='Frame')

# Plot the output polygon
plt.plot(output_x + [output_x[0]], output_y + [output_y[0]], 'g-', label='Output Polygon')

# Add labels and title to the plot
plt.legend()
plt.xlabel('X')
plt.ylabel('Y')
plt.title('Polygon Clipping')
plt.grid(True)

# Display the plot
plt.show()


# Problem **11**
Run this code in any other IDE not in Colab